In [1]:
import gensim
import import_ipynb
import itertools
import nltk
import NLP_preprocessing as pre
import numpy as np
import pandas as pd
import tensorflow as tf
import time

from gensim import corpora
from keras.callbacks import TensorBoard, EarlyStopping  
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.layers import SimpleRNN, LSTM, GRU, Masking, Reshape, Bidirectional
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

importing Jupyter notebook from NLP_preprocessing.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Load data

In [2]:
data = pd.read_excel('отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
max_words = 20000
max_len = 78
num_classes = 5

epochs = 20
batch_size = 16
print_batch_n = 100

model_score = pd.DataFrame(columns=['Model', 'Loss', 'Accuracy', 'Time (min)'])

#### Preprocessing

In [4]:
%%time
tqdm.pandas()
data['Content_pre'] = data['Content'].progress_apply(pre.text_preprocess_light)
data.insert(1, 'Content_token', [pre.word_tokenize(text) for text in data['Content_pre']], allow_duplicates=True)
stem_content_token =[]
for tokens in tqdm(data['Content_token']):
    stem_token =  [pre.ps().stem(token) for token in tokens]
    stem_content_token.append(stem_token)
data.insert(2, 'Content_stemmed', stem_content_token, allow_duplicates=True)
lemma_content_token =[]
for tokens in tqdm(data['Content_stemmed']):
    lemma_token =  [pre.lem().lemmatize(token) for token in tokens]
    lemma_content_token.append(lemma_token)
data.insert(3, 'Content_lemmatized', lemma_content_token, allow_duplicates=True)

100%|█████████████████████████████████████████████████████████████████████████| 20659/20659 [00:01<00:00, 10962.66it/s]

Wall time: 25.5 s


In [5]:
data

,Rating,Content_token,Content_stemmed,Content_lemmatized,Content,Date,Content_pre
0,5,[],[],[],It just works!,2017-08-14,
1,4,"[целое, удобноной, приложение, минус, хотеть, ...","[целое, удобноной, приложение, минус, хотеть, ...","[целое, удобноной, приложение, минус, хотеть, ...",В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть большо...
2,5,[отлично],[отлично],[отлично],Отлично все,2017-08-14,отлично
3,5,"[зависать, работа, антивирус, ранее, пользоват...","[зависать, работа, антивирус, ранее, пользоват...","[зависать, работа, антивирус, ранее, пользоват...",Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать работа антивирус ранее пользоваться н...
4,5,"[удобно, работать, быстро]","[удобно, работать, быстро]","[удобно, работать, быстро]","Очень удобно, работает быстро.",2017-08-14,удобно работать быстро
...,...,...,...,...,...,...,...
20654,1,"[шляпа, роот, право, бесполезный, прога, разра...","[шляпа, роот, право, бесполезный, прога, разра...","[шляпа, роот, право, бесполезный, прога, разра...","Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,шляпа роот право бесполезный прога разраб охре...
20655,5,[],[],[],Ок,2017-06-01,
20656,4,[довольный],[довольный],[довольный],Доволен,2017-06-01,довольный
20657,1,"[песопаснастя, рута, нужын]","[песопаснастя, рута, нужын]","[песопаснастя, рута, нужын]","Песопаснасть, рут ни нужын",2017-06-01,песопаснастя рута нужын


In [6]:
# Словарь уникальных токенов
dictionary = corpora.Dictionary(data['Content_lemmatized'])

In [7]:
# Выкидывание редких слов
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('content.dict')
len(dictionary.keys())

1101

In [8]:
tokens = [dictionary[i] for i in range(len(dictionary))]
tokens

['большой',
 'данные',
 'доступ',
 'минус',
 'ограниченный',
 'пользоваться',
 'приложение',
 'режим',
 'хотеть',
 'целое',
 'отлично',
 'антивирус',
 'зависать',
 'нормально',
 'работа',
 'ранее',
 'быстро',
 'работать',
 'удобно',
 'норма',
 'удобный',
 'устраивать',
 'банкомат',
 'вечно',
 'отличие',
 'тупить',
 'чётко',
 'нельзя',
 'уведомление',
 'удалять',
 'звонок',
 'книга',
 'один',
 'слово',
 'замечательный',
 'штука',
 'отличный',
 'практично',
 'оплата',
 'перевод',
 'платёж',
 'вообще',
 'карта',
 'пополнять',
 'жаль',
 'история',
 'хороший',
 'лишний',
 'полезный',
 'висеть',
 'дважды',
 'загрузка',
 'заход',
 'пароль',
 'первый',
 'список',
 'требовать',
 'запуститься',
 'сбербанк',
 'часы',
 'супер',
 'комфортно',
 'поиск',
 'служба',
 'молодец',
 'круто',
 'взгляд',
 'нарекание',
 'недолго',
 'мнеудобный',
 'ура',
 'долгий',
 'идти',
 'использование',
 'исправт',
 'конец',
 'неустанавливаться',
 'обновление',
 'повторный',
 'попытка',
 'последний',
 'программа',
 'скач

In [9]:
voc = {v: k for k, v in dict(enumerate(tokens, 1)).items()}

In [10]:
def text_to_sequence(tokens, maxlen):
    result = []
    for word in tokens:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [11]:
data_train = np.asarray([text_to_sequence(text, max_len) for text in data['Content_lemmatized']], dtype=np.int32)
data_train

array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    6,    5,    8],
       [   0,    0,    0, ...,    0,    0,   11],
       ...,
       [   0,    0,    0, ...,    0,    0,  123],
       [   0,    0,    0, ...,    0,    0,  294],
       [   0,    0,    0, ..., 1056,  210, 1024]])

In [12]:
count = 0
max_count = 0
for i in range(len(data_train)):
    for j in range(len(data_train[i])):
        if data_train[i][j] != 0:
            count += 1
        if max_count < count:
            max_count = count
    count = 0
print(max_count)

77


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [14]:
X_train.shape

(14461, 78)

In [15]:
le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [16]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Gpu acceleration and reduced memory usage

In [17]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.config.set_soft_device_placement(True)

#### CNN Model with default Embedding layer

In [18]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dropout(rate=0.25))
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [19]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=2,
          verbose=1,
          validation_split=0.1)
end = (time.time() - start) / 60

Epoch 1/2
814/814 [==============================] - 15s 16ms/step - loss: 0.7519 - accuracy: 0.7374 - val_loss: 0.6928 - val_accuracy: 0.7623
Epoch 2/2
814/814 [==============================] - 15s 18ms/step - loss: 0.6601 - accuracy: 0.7738 - val_loss: 0.6664 - val_accuracy: 0.7699


In [21]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'CNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 2s 4ms/step - loss: 0.6789 - accuracy: 0.7614
Test score: 0.6788845062255859
Test accuracy: 0.7613746523857117


#### SimpleRNN Model

In [22]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 103s 140ms/step - loss: 0.7798 - accuracy: 0.7292 - val_loss: 0.6726 - val_accuracy: 0.7681
Epoch 2/10
723/723 [==============================] - 103s 143ms/step - loss: 0.6640 - accuracy: 0.7705 - val_loss: 0.6452 - val_accuracy: 0.7764


In [23]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'SimpleRNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 8s 20ms/step - loss: 0.6713 - accuracy: 0.7643
Test score: 0.6713127493858337
Test accuracy: 0.7642788290977478


#### CNN Model with default Embedding layer -> Simple RNN Model

In [24]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(rate=0.25))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 90s 122ms/step - loss: 0.7631 - accuracy: 0.7438 - val_loss: 0.6961 - val_accuracy: 0.7605
Epoch 2/10
723/723 [==============================] - 94s 130ms/step - loss: 0.6730 - accuracy: 0.7698 - val_loss: 0.6692 - val_accuracy: 0.7774


In [25]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'CNN->SimpleRNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 8s 20ms/step - loss: 0.6851 - accuracy: 0.7649
Test score: 0.6850776076316833
Test accuracy: 0.764924168586731


#### Simple RNN Model -> CNN Model with default Embedding layer

In [26]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((128,1)))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 105s 143ms/step - loss: 0.8254 - accuracy: 0.7234 - val_loss: 0.7450 - val_accuracy: 0.7608
Epoch 2/10
723/723 [==============================] - 99s 137ms/step - loss: 0.6976 - accuracy: 0.7606 - val_loss: 0.6681 - val_accuracy: 0.7736


In [27]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'SimpleRNN->CNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 11s 27ms/step - loss: 0.6973 - accuracy: 0.7593
Test score: 0.6972811222076416
Test accuracy: 0.7592771649360657


#### LSTM Model

In [28]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 171s 229ms/step - loss: 0.7707 - accuracy: 0.7415 - val_loss: 0.6649 - val_accuracy: 0.7705
Epoch 2/10
723/723 [==============================] - 181s 250ms/step - loss: 0.6373 - accuracy: 0.7789 - val_loss: 0.6466 - val_accuracy: 0.7802


In [29]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'LSTM',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 22s 57ms/step - loss: 0.6733 - accuracy: 0.7654
Test score: 0.6732935309410095
Test accuracy: 0.7654082179069519


#### Bidirectional LSTM Model

In [30]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))

forward_layer = LSTM(128, return_sequences=True)
backward_layer = LSTM(128, activation='relu', return_sequences=True,
                      go_backwards=True)

model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                        input_shape=(256,1)))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 327s 447ms/step - loss: 138565.5000 - accuracy: 0.7308 - val_loss: 4.7980 - val_accuracy: 0.7553
Epoch 2/10
723/723 [==============================] - 323s 447ms/step - loss: 941.8324 - accuracy: 0.7541 - val_loss: 1.0628 - val_accuracy: 0.7636


In [31]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'BidirectionalLSTM',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 47s 120ms/step - loss: 0.7539 - accuracy: 0.7515
Test score: 0.7539249062538147
Test accuracy: 0.7515327334403992


#### CNN -> LSTM Model

In [32]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(rate=0.25))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 69s 91ms/step - loss: 0.7419 - accuracy: 0.7507 - val_loss: 0.6569 - val_accuracy: 0.7784
Epoch 2/10
723/723 [==============================] - 67s 93ms/step - loss: 0.6430 - accuracy: 0.7765 - val_loss: 0.6501 - val_accuracy: 0.7798


In [33]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'CNN->LSTM',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 12s 30ms/step - loss: 0.6696 - accuracy: 0.7657
Test score: 0.6696120500564575
Test accuracy: 0.7657308578491211


#### CNN -> Bidirectional LSTM Model

In [34]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))

forward_layer = LSTM(128, return_sequences=True)
backward_layer = LSTM(128, activation='relu', return_sequences=True,
                      go_backwards=True)

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                        input_shape=(256,1)))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 201s 274ms/step - loss: 10.1268 - accuracy: 0.7411 - val_loss: 0.6847 - val_accuracy: 0.7691
Epoch 2/10
723/723 [==============================] - 186s 258ms/step - loss: 0.6541 - accuracy: 0.7754 - val_loss: 0.6419 - val_accuracy: 0.7767


In [35]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'CNN->BidirectionalLSTM',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 21s 54ms/step - loss: 0.6762 - accuracy: 0.7612
Test score: 0.6762344837188721
Test accuracy: 0.7612133026123047


#### LSTM -> CNN Model

In [36]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))

model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((128,1)))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 179s 240ms/step - loss: 0.8364 - accuracy: 0.7196 - val_loss: 0.6861 - val_accuracy: 0.7729
Epoch 2/10
723/723 [==============================] - 164s 227ms/step - loss: 0.6633 - accuracy: 0.7720 - val_loss: 0.6544 - val_accuracy: 0.7750


In [37]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'LSTM->CNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 25s 65ms/step - loss: 0.6845 - accuracy: 0.7561
Test score: 0.6845206022262573
Test accuracy: 0.7560503482818604


#### Bidirectional LSTM -> CNN Model

In [38]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))

forward_layer = LSTM(128, return_sequences=True)
backward_layer = LSTM(128, activation='relu', return_sequences=True,
                      go_backwards=True)

model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                        input_shape=(256,1)))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())

model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.add(Dropout(rate=0.25))
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 311s 425ms/step - loss: 2.8814 - accuracy: 0.5850 - val_loss: 1.5351 - val_accuracy: 0.7636
Epoch 2/10
723/723 [==============================] - 300s 414ms/step - loss: 2.5197 - accuracy: 0.5899 - val_loss: 1.4898 - val_accuracy: 0.7646


In [39]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'BidirectionalLSTM->CNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 40s 102ms/step - loss: 1.5992 - accuracy: 0.7520
Test score: 1.5992189645767212
Test accuracy: 0.7520167827606201


#### GRU Model

In [40]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 144s 192ms/step - loss: 0.7656 - accuracy: 0.7461 - val_loss: 0.6497 - val_accuracy: 0.7781
Epoch 2/10
723/723 [==============================] - 146s 202ms/step - loss: 0.6310 - accuracy: 0.7801 - val_loss: 0.6192 - val_accuracy: 0.7895


In [41]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'GRU',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 16s 42ms/step - loss: 0.6514 - accuracy: 0.7702
Test score: 0.651425302028656
Test accuracy: 0.7702484726905823


#### CNN -> GRU Model

In [42]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(rate=0.25))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 49s 65ms/step - loss: 0.7311 - accuracy: 0.7519 - val_loss: 0.6443 - val_accuracy: 0.7784
Epoch 2/10
723/723 [==============================] - 51s 71ms/step - loss: 0.6389 - accuracy: 0.7787 - val_loss: 0.6424 - val_accuracy: 0.7805


In [43]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'CNN->GRU',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 3s 8ms/step - loss: 0.6793 - accuracy: 0.7628
Test score: 0.6793103814125061
Test accuracy: 0.7628267407417297


#### GRU -> CNN Model

In [44]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))

model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((128,1)))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

start = time.time()
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])
end = (time.time() - start) / 60

Epoch 1/10
723/723 [==============================] - 143s 192ms/step - loss: 0.7750 - accuracy: 0.7363 - val_loss: 0.6810 - val_accuracy: 0.7753
Epoch 2/10
723/723 [==============================] - 144s 199ms/step - loss: 0.6595 - accuracy: 0.7738 - val_loss: 0.6489 - val_accuracy: 0.7802


In [45]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

model_score = model_score.append({
    'Model': 'GRU->CNN',
    'Loss': score[0],
    'Accuracy': score[1],
    'Time (min)': end
}, ignore_index=True)

388/388 [==============================] - 19s 49ms/step - loss: 0.6801 - accuracy: 0.7631
Test score: 0.6801000237464905
Test accuracy: 0.7631493806838989


In [46]:
model_score.sort_values(by='Accuracy', ascending=False)

,Model,Loss,Accuracy,Time (min)
10,GRU,0.651425,0.770248,4.823787
6,CNN->LSTM,0.669612,0.765731,2.265920
4,LSTM,0.673294,0.765408,5.855786
2,CNN->SimpleRNN,0.685078,0.764924,3.065910
1,SimpleRNN,0.671313,0.764279,3.438086
12,GRU->CNN,0.680100,0.763149,4.782033
11,CNN->GRU,0.679310,0.762827,1.679360
0,CNN,0.678885,0.761375,0.496027
7,CNN->BidirectionalLSTM,0.676234,0.761213,6.451185
3,SimpleRNN->CNN,0.697281,0.759277,3.404856


In [47]:
model_score.to_csv('model_score.csv')